In [121]:
import glob
import pandas as pd
from datetime import datetime

path = 'homepage_csvs/test/'
path = 'homepage_csvs/chaturbate/'

# read file names from directory
csv_files = glob.glob(path + '*.csv')

# iterate over files to create a dict of performers and their sessions
performerlist = {}
for filename in sorted(csv_files):
    
    df = pd.read_csv(filename)
    fileparts = filename.split('_')
    date_string = fileparts[2] + ' ' + fileparts[3]
    
    date = datetime.strptime(date_string, '%Y-%m-%d %H-%M-%S')
    
    for name in df['name']:
        if name not in performerlist:
            performerlist[name] = set()
        performerlist[name].add(date)
            
#print(len(performerlist))
#print(performerlist)


In [124]:
sessions = []
sid = 0

for name in performerlist:
    
    #if name != 'freya_montclair':
       #continue
    
    performer = performerlist[name]
    #print(name, sorted(performer))
    
    last_stamp = None
    counter = 0
    
    for stamp in sorted(performer):
        
        if last_stamp == None:
            
            ns = {}
            ns['id'] = sid
            sid += 1
            
            ns['name'] = name
            ns['start'] = stamp
        
            last_stamp = stamp
        
        else:
            
            delta = stamp - last_stamp
            #print(delta.total_seconds())
            
            if delta.total_seconds() > 3700 and counter < len(performer):
                ns['end'] = last_stamp
                ns['duration'] = (last_stamp - ns['start']).total_seconds()
                sessions.append(ns)
                
                ns = {}
                ns['id'] = sid
                sid += 1
                ns['name'] = name
                ns['start'] = stamp
                
            last_stamp = stamp
            

        if counter == len(performer)-1:
            ns['end'] = stamp
            ns['duration'] = (stamp - ns['start']).total_seconds()
            sessions.append(ns)
            last_stamp = None

        
        counter += 1
        
df = pd.DataFrame.from_dict(sessions)
df.to_csv('sessions.csv',index=False)

#sessions
